In [ ]:
!pip install pandas datasets

In [ ]:
!pip install transformers torch

In [ ]:
!pip install xformers trl peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 19.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 899.7/899.7 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 594.3/594.3 MB 1.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 47.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 88.0/88.0 MB 29.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 954.8/954.8 kB 41.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.1/193.1 MB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 63.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.6/63.6 MB 30.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 267.5/267.5 MB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.2/288.2 MB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.3/39.3 MB 50.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 

In [ ]:
import sys
sys.version

'3.12.12 (main, Oct 10 2025, 08:52:57) [GCC 11.4.0]'

In [ ]:
import json
from datasets import Dataset
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer,
    DataCollatorForLanguageModeling,
    BitsAndBytesConfig
)
from peft import LoraConfig, get_peft_model, prepare_model_for_kbit_training
import torch

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
elif torch.mps.is_available():
    device = "mps"
else:
    device = "cpu"

print("We would be using this device:", device)

We would be using this device: cuda


# Loading the TL;DR Dataset

we have saved the TL;DR Dataset in the JSONL format

In [ ]:
# Load JSONL data (TL;DR Dataset)
print("\n[1/8] Loading dataset...")
data = []
with open("proc_tldr.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

dataset = Dataset.from_dict({
    "messages": [item["messages"] for item in data]
})

print(f"✓ Loaded {len(dataset)} examples")


[1/8] Loading dataset...
✓ Loaded 6944 examples


# Preparing the Train and Test Dataset

we have decided to split 90% for training and 10% for testing

In [ ]:
# Split dataset
split_dataset = dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]
print(f"✓ Train: {len(train_dataset)} | Val: {len(val_dataset)}")

✓ Train: 6249 | Val: 695


In [ ]:
print("\n[2/8] Configuring 8-bit quantization...")
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16,
)


[2/8] Configuring 8-bit quantization...


# Model

## Note

we have login inside hugging face and make sure we request for access to use the LLaMA 3.2 3B model

In [ ]:
!huggingface-cli login

⚠️  Warning: 'huggingface-cli login' is deprecated. Use 'hf auth login' instead.

    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: read).
The token `YTA-DEV` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/hu

In [ ]:
# Load model and tokenizer
print("\n[3/8] Loading LLaMA 3.2 3B model...")
model_name = "meta-llama/Llama-3.2-3B-Instruct"

tokenizer = AutoTokenizer.from_pretrained(
    model_name,
    use_fast=True
)

# LLaMA models usually do not have a pad token by default
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=bnb_config,
    device_map="auto"
)

print("✓ Model loaded in 8-bit")
print("✓ Model size: ~3B parameters")



[3/8] Loading LLaMA 3.2 3B model...


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/54.5k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/1.46G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/4.97G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/189 [00:00<?, ?B/s]

✓ Model loaded in 8-bit
✓ Model size: ~3B parameters


In [ ]:
# Prepare for k-bit training
print("\n[4/8] Preparing model for QLoRA...")
model = prepare_model_for_kbit_training(
    model,
    use_gradient_checkpointing=True
)

# (Optional but recommended) disable cache during training
model.config.use_cache = False

# Configure LoRA
lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=[
        "q_proj",
        "k_proj",
        "v_proj",
        "o_proj",
        "gate_proj",
        "up_proj",
        "down_proj",
    ],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)


[4/8] Preparing model for QLoRA...


In [ ]:
model = get_peft_model(model, lora_config)
print("✓ LoRA adapters added")
model.print_trainable_parameters()

# Tokenization function with proper chat formatting
print("\n[5/8] Preparing tokenization...")

✓ LoRA adapters added
trainable params: 24,313,856 || all params: 3,237,063,680 || trainable%: 0.7511

[5/8] Preparing tokenization...


In [ ]:
def format_and_tokenize(example, force=4096):
    # Apply LLaMA chat template
    formatted_text = tokenizer.apply_chat_template(
        example["messages"],
        tokenize=False,
        add_generation_prompt=False
    )

    # Tokenize (NO padding, NO manual labels)
    tokenized = tokenizer(
        formatted_text,
        truncation=True,
        max_length=force,   # IMPORTANT: increase for long transcripts
        padding=False,
    )

    return tokenized


In [ ]:
# Apply tokenization
print("✓ Tokenizing train dataset...")
train_dataset = train_dataset.map(
    format_and_tokenize,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing train"
)


✓ Tokenizing train dataset...


Tokenizing train:   0%|          | 0/6249 [00:00<?, ? examples/s]

In [ ]:
print("✓ Tokenizing validation dataset...")
val_dataset = val_dataset.map(
    format_and_tokenize,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing validation"
)

✓ Tokenizing validation dataset...


Tokenizing validation:   0%|          | 0/695 [00:00<?, ? examples/s]

In [ ]:
# Show sample stats (pre-training)
sample = train_dataset[0]

print("\n✓ Sample stats:")
print(f"  - Input length: {len(sample['input_ids'])} tokens")
print(f"  - Attention tokens: {sum(sample['attention_mask'])} tokens")
print(f"  - Truncated: {'Yes' if len(sample['input_ids']) == 4096 else 'No'}")



✓ Sample stats:
  - Input length: 1504 tokens
  - Attention tokens: 1504 tokens
  - Truncated: No


In [ ]:
print("\n[6/8] Configuring training...")
training_args = TrainingArguments(
    output_dir="./llama3.2-3b-qlora-summary",
    num_train_epochs=1,

    # Memory-optimized batch settings
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=8,

    # Learning settings
    learning_rate=2e-4,
    weight_decay=0.01,
    warmup_steps=100,
    max_grad_norm=1.0,

    # Evaluation and saving
    eval_strategy="steps",
    eval_steps=100,
    save_steps=100,
    save_total_limit=2,
    load_best_model_at_end=True,

    # Optimization
    bf16=True,
    optim="paged_adamw_8bit",
    gradient_checkpointing=True,

    # Efficiency
    group_by_length=True,
    dataloader_num_workers=4,

    # Logging
    logging_steps=10,
    report_to="none",
)


[6/8] Configuring training...


In [ ]:
print("✓ Training configuration:")
print(f"  - Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"  - Total training steps: ~{len(train_dataset) * 3 // (training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps)}")
print(f"  - Learning rate: {training_args.learning_rate}")

✓ Training configuration:
  - Effective batch size: 16
  - Total training steps: ~1171
  - Learning rate: 0.0002


In [ ]:
# Data collator with dynamic padding
print("\n[7/8] Creating data collator...")
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=False,
    pad_to_multiple_of=8  # Efficient padding for GPU
)


[7/8] Creating data collator...


In [ ]:
# Create trainer
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    data_collator=data_collator,
)


In [ ]:
# Start training
print("\n[8/8] Starting training...")
print("=" * 60)
print("TRAINING IN PROGRESS")
print("=" * 60)

trainer.train()


[8/8] Starting training...
TRAINING IN PROGRESS


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")


Step,Training Loss,Validation Loss
100,2.164900,2.286720
200,2.145200,2.253277
300,2.158900,2.239438


/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:123: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during

TrainOutput(global_step=391, training_loss=2.296681910219705, metrics={'train_runtime': 4714.9168, 'train_samples_per_second': 1.325, 'train_steps_per_second': 0.083, 'total_flos': 1.2578198791033651e+17, 'train_loss': 2.296681910219705, 'epoch': 1.0})

In [ ]:
# Save final model
print("\n" + "=" * 60)
print("TRAINING COMPLETE")
print("=" * 60)
print("\nSaving model...")
trainer.save_model("./llama3.2-3b-qlora-summary")
tokenizer.save_pretrained("./llama3.2-3b-qlora-summary")


TRAINING COMPLETE

Saving model...


('./llama3.2-3b-qlora-summary/tokenizer_config.json',
 './llama3.2-3b-qlora-summary/special_tokens_map.json',
 './llama3.2-3b-qlora-summary/chat_template.jinja',
 './llama3.2-3b-qlora-summary/tokenizer.json')

In [ ]:
!zip -r ./llama_3b_3_2.zip ./llama3.2-3b-qlora-summary

  adding: llama3.2-3b-qlora-summary/ (stored 0%)
  adding: llama3.2-3b-qlora-summary/tokenizer.json (deflated 85%)
  adding: llama3.2-3b-qlora-summary/adapter_config.json (deflated 58%)
  adding: llama3.2-3b-qlora-summary/tokenizer_config.json (deflated 96%)
  adding: llama3.2-3b-qlora-summary/README.md (deflated 65%)
  adding: llama3.2-3b-qlora-summary/chat_template.jinja (deflated 71%)
  adding: llama3.2-3b-qlora-summary/special_tokens_map.json (deflated 63%)
  adding: llama3.2-3b-qlora-summary/checkpoint-300/ (stored 0%)
  adding: llama3.2-3b-qlora-summary/checkpoint-300/tokenizer.json (deflated 85%)
  adding: llama3.2-3b-qlora-summary/checkpoint-300/adapter_config.json (deflated 58%)
  adding: llama3.2-3b-qlora-summary/checkpoint-300/scheduler.pt (deflated 61%)
  adding: llama3.2-3b-qlora-summary/checkpoint-300/tokenizer_config.json (deflated 96%)
  adding: llama3.2-3b-qlora-summary/checkpoint-300/README.md (deflated 65%)
  adding: llama3.2-3b-qlora-summary/checkpoint-300/chat_temp

# Loading Custom Dataset

In [ ]:
# Load JSONL data (Custom Dataset)
print("\n2-[1/8] Loading dataset...")
data = []
with open("custom_dataset.jsonl", "r") as f:
    for line in f:
        data.append(json.loads(line))

custom_dataset = Dataset.from_dict({
    "messages": [item["messages"] for item in data]
})

print(f"✓ Loaded {len(custom_dataset)} examples")


2-[1/8] Loading dataset...
✓ Loaded 1102 examples


In [ ]:
# Split dataset
split_dataset = custom_dataset.train_test_split(test_size=0.1, seed=42)
train_dataset = split_dataset["train"]
val_dataset = split_dataset["test"]
print(f"✓ Train: {len(train_dataset)} | Val: {len(val_dataset)}")

✓ Train: 991 | Val: 111


In [ ]:
model.print_trainable_parameters()

trainable params: 24,313,856 || all params: 3,237,063,680 || trainable%: 0.7511


In [ ]:
format_and_tokenize_for_cds = lambda x: format_and_tokenize(x, force=2 ** 15)

In [ ]:
# Apply tokenization
print("✓ Tokenizing train dataset...")
train_dataset = train_dataset.map(
    format_and_tokenize_for_cds,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing train"
)


✓ Tokenizing train dataset...


Tokenizing train:   0%|          | 0/991 [00:00<?, ? examples/s]

In [ ]:
print("✓ Tokenizing validation dataset...")
val_dataset = val_dataset.map(
    format_and_tokenize_for_cds,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing validation"
)

✓ Tokenizing validation dataset...


Tokenizing validation:   0%|          | 0/111 [00:00<?, ? examples/s]

In [ ]:
# Show sample stats (pre-training)
sample = train_dataset[0]

print("\n✓ Sample stats:")
print(f"  - Input length: {len(sample['input_ids'])} tokens")
print(f"  - Attention tokens: {sum(sample['attention_mask'])} tokens")
print(f"  - Truncated: {'Yes' if len(sample['input_ids']) == 2 ** 15 else 'No'}")



✓ Sample stats:
  - Input length: 6141 tokens
  - Attention tokens: 6141 tokens
  - Truncated: No


In [ ]:
well_note = custom_dataset.map(
    format_and_tokenize_for_cds,
    remove_columns=["messages"],
    batched=False,
    desc="Tokenizing train"
)

Tokenizing train:   0%|          | 0/1102 [00:00<?, ? examples/s]

In [ ]:
sie = []
note = []
for index, _ in enumerate(well_note):
  if len(_['input_ids']) == 2 ** 15:
    note.append(index)
  sie.append(len(_['input_ids']))


px.scatter(sie)

In [ ]:
import regex

pattern = r"USER PROMPT:\s*([\s\S]*?)(?:\nVIDEO TRANSCRIPTS|\Z)"

for index in note:
    result = regex.findall(pattern, custom_dataset[index]["messages"][1]["content"])
    print(result[0])

In [ ]:
type(custom_dataset[index])

dict